# Shock Tube and Tunnel Calculator
-------------------------
Objective: calculate the conditions experienced by the user-specified driven gas in a typical shock tube/tunnel experimental test. While the shock tube is always assumed to be in equilibrium, the nozzle exit parameters are calculated assuming three different isentropic conditions: frozen, equilibrium and an instantaneous transition from equilibrium to frozen flow at user-definied Mach number position.

How: through Rankine-Hugoniot relations and Cantera, a suite software that provides us equilibrium gas parameters.   


Required libraries:
 - Numpy 
 - Scipy
 - Matplotlib

 - Cantera:<br>
*'Cantera is a suite of object-oriented software tools for problems involving chemical kinetics, thermodynamics, and/or transport processes. Cantera provides types (or classes) of objects representing phases of matter, interfaces between these phases, reaction managers, time-dependent reactor networks, and steady one-dimensional reacting flows.Cantera is currently used for applications including combustion, detonations, electrochemical energy conversion and storage, fuel cells, batteries, aqueous electrolyte solutions, plasmas, and thin film deposition.'* [(Cantera Tutorial)](https://www.cerfacs.fr/cantera/docs/tutorials/CANTERA_HandsOn.pdf)<br>
[Getting Started With Cantera](https://cantera.org/tutorials/index.html#cantera-next-steps)<br>
[Installing Cantera](https://cantera.org/install/index.html)<br>

 - iPyWidgets<br>
*'...[widgets] can be embedded in the notebook and provide a user friendly interface to collect the user input and see the impact the changes have on the data/results, without having to interact with the code...'*
[(How to create dynamic dashboards using ipywidgets)](https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916)<br>
[Simple Widget Introduction](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html)<br>
[Installing iPyWidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html)

In [1]:
%matplotlib inline
import numpy as np
import cantera as ct
import matplotlib.pylab as plt
from scipy.optimize import root 
from scipy.optimize import brenth
from Quenching_Data import Fluorescence_Lifetime as tfl

#from ipywidgets import interactive_output
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import time

import pandas as pd

#### List of Symbols: 
- $Ar$: nozzle area ratio
- $a$: sound speed 
- $dD$: freestream stand-off distance ratio
- $g$: heat capacity ratio
- $h$: specific enthalpy 
- $M$: Mach number
- $mu$: viscosity
- $p$: pressure 
- $Pto$: Pitot pressure
- $r$: density 
- $Rg$: individual gas constant 
- $s$: specific entropy 
- $t$: temperature
- $tau$: *NO(A,v=0)* fluorescence lifetime
- $tc$: thermal conductivity
- $u$: speed
- $vtau$: characteristic fluorescence lenght

(all parameters in SI units)

#### List of suffix: 
- $1$: initial driven condition in `Calc_Ms` and `Shock12` functions; and freestream condition in `Frozen_Area`, `Equilibrum_Area` and `Eq_At_M_Area`. 
- $2$: shocked condition in `Shock12` function; and freestream post normal shock condition in `Frozen_Area`, `Equilibrum_Area` and `Eq_At_M_Area`. 
- $5$: reflected condition 
- $5E$: reservoir condition - calculated from measured reflected pressure
- $.X['M']$: molar fraction of M molecular specie
- $s$: relative to incident shock wave 
- $r$: relative to reflected shock wave



#### The Shock Tube Equation: Driver/Driven vs Ms
The shock tube calculation begins with the `Calc_Ms` function, that uses the following equations to relate the initial shock tube condition (i.e. driver and driven conditions) to the incident shock wave of Mach number  <i>Ms</i> that follows an ideal instantaneous diaphragm rupture [1]:

$$ \frac{p_2}{p_1} =1 + \frac{2g_1}{g_1+1}(M_s^2 - 1) $$
$$\frac{p_2}{p_1}\left[1 - \frac{(g_4 - 1)(a_1/a_4)(p_2/p_1 -1)}{\sqrt{ 2g_1[2g_1 + (g_1+1)(p_2/p_1 -1)] } } \right]^{\frac{-2g_4}{g_4 - 1}}  = \frac{p_4}{p_1}$$

The `Calc_Ms` function returns the difference between both sides of the second equation and an objective function is called in the `Input` function to find the <i>Ms</i> value that satisfies the above relations. 

[1] Anderson, J. D. (1990). Shock Tube Relations. <i>Modern compressible flow: with historical perspective.</i> Tata McGraw-Hill Education.

In [2]:
def Calc_Ms(Ms,  p1,a1,g1,g4,a4,p4):             
    p21     = ( 2.*g1*Ms*Ms - (g1-1) )/(g1+1) #p2/p1
    sqr     = ( 2*g1*(  2*g1+(g1+1)*(p21-1)  ) )**0.5 
    return (p4/p1) - p21*(1. - (g4-1)*(a1/a4)*(p21 -1)/sqr  )**(-2.*g4/(g4-1.))

#### The Shocked Condition

Knowing the initial driven condition and incident shock speed, the shocked condition, induced by the moving incident shock wave generated by the diaphragm bursting, can be calculated assuming an adiabatic, inviscid, constant area duct, and equilibrium flow by the following iterative steps [see 2,3]:


0. Specify the initial driven condition ($r_1$ and $h_1$) from initial pressure $p_1$ and temperature $t_1$ by using Cantera Suite;
1. Guess a shocked gas density greater than the initial gas density: $r_2 = 10 r_1$;
2. Calculate pressure $p_2$ and enthalpy $h_2$ from initial data and guessed gas density:
$$p_2 = p_1 + r_1 u_s^2\left(1 - \frac{r_1}{r_2}\right), $$
$$h_2 = h_1 + \frac{u_s^2}{2}\left[ 1 - \left( \frac{r_1}{r_2} \right)^2 \right];$$
3. Calculate a new density $r_2$ from a equation of state: 
$$ r_2 = r_2(h_2,p_2);$$
Here the Cantera package is used to calculate the shocked density from the local pressure and enthalpy.

4. Repeat steps 2 and 3 until convergence criteria is achieved; 

5. Specify shocked condition (as step 0) from pressure and enthalpy obtained in step 2 by using Cantera. 

The equations in step 2 correspond to the Rankine-Hugoniot relations for conservation of momentum and energy in the shock wave reference after replacing $u_2$ by $u_s(1 -r_1/r_2)$.  

The `Shock12` function has driven temperature $T_1$, pressure $p_1$, and incident shock speed $u_s$ as input parameters and returns shocked parameters as entropy, viscosity and mole fraction. 

----------------------------------------------------------------

[2] Loubsky, W. J., & Reller Jr, J. D. (1966). Appendix A. <i>[Analysis of tailored-interface operation of shock tubes with helium-driven planetary gases](https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/19660020190.pdf)</i>, Nasa TN D-3495.

[3] Anderson Jr, J. D. (2006).Chapter: Inviscid High-Temperature Equilibrium Flows. <i>Hypersonic and high-temperature gas dynamics</i>. American Institute of Aeronautics and Astronautics. pp 605.

In [3]:
def Shock12(t1,p1,us):
    Air_1.TP = t1,p1; Air_1.equilibrate('TP',loglevel=0,rtol=1e-4) #step 0
    Rg       = ct.gas_constant/Air_1.mean_molecular_weight         #step 0
    g        = Air_1.cp_mole/Air_1.cv_mole                         #step 0
    r1       = Air_1.density                                       #step 0
    h1       = Air_1.enthalpy_mass                                 #step 0
    Ms       = us/np.sqrt(g*Rg*t1)                                 #step 0

    Res = 1.
    r2 = r1*10                                                            #step 1
    while (Res > 1e-4):                                                   #step 4
        p2 = p1 +  r1*us*us*(1.0 - r1/r2        )                         #step 2
        h2 = h1 + 0.5*us*us*(1.0 - (r1/r2)**2.0 )                         #step 2

        Air_1.HP = h2,p2; Air_1.equilibrate('HP',loglevel=0,rtol=1e-4)    #step 3
        Res      = np.abs(r2 - Air_1.density)/r2                          #step 3
        r2       = Air_1.density                                          #step 3 
        
    u2 = us*(1.-r1/r2)                                                  #step 5 
    t2 = Air_1.T                                                        #step 5
    s2 = Air_1.entropy_mass                                             #step 5    
    g2 = Air_1.cp_mole/Air_1.cv_mole                                    #step 5
    a2 = np.sqrt(g2*(ct.gas_constant/Air_1.mean_molecular_weight)*t2)   #step 5
    r2 = Air_1.density                                                  #step 5
    mu2 = Air_1.viscosity #Pa.s                                         #step 5
    tc2 = Air_1.thermal_conductivity #W/(m.K)                           #step 5
    return u2,p2,t2,r2,g2,a2,h2,s2,mu2,tc2,Air_1['N2'].X,Air_1['O2'].X,Air_1['NO'].X,Air_1['O'].X

#### The Reflected Condition

The procedure followed in `Shock25` function is analogous to that shown in `Shock12`, however, the following assumes the reflected shock wave is such that its intensity brings the shocked gas to rest. Thus, the steps followed in this function are [3]:  

1. Guess a reflected gas density greater than the shocked gas density: $r_5 = 10 r_2$;
2. Calculate reflected shock speed, pressure $p_5$ and enthalpy $h_5$ from shocked condition and guessed density:
$$u_r = \frac{u_2}{r_5/r_2  -  1},$$
$$p_5 = p_2 + r_2 (u_r+u_2)^2\left(1 - \frac{r_2}{r_5}\right), $$
$$h_5 = h_2 + \frac{p_5-p_2}{2}\left( \frac{1}{r_2} + \frac{1}{r_5}   \right);$$
3. Calculate a new density $r_5$ from a equation of state: 
$$ r_5 = r_5(h_5,p_5);$$
Here the Cantera package is used to calculate the reflected density from the local pressure and enthalpy.

4. Repeat steps 2 and 3 until convergence criteria is achieved; 

5. Specify reflected condition from pressure and enthalpy obtained in step 2. 

The three equations in step 2 are obtained from the equations of conservation of mass, momentum and energy in the reflected shock reference system.

The `Shock25` function has shocked temperature, pressure, density, speed, and sound speed as input parameters and returns reflected parameters as entropy, viscosity and mole fraction. 

In [4]:
def Shock25(t2,p2,r2,u2,a2):
    Air_1.TP = t2,p2; Air_1.equilibrate('TP',loglevel=0,rtol=1e-4)
    h2 = Air_1.enthalpy_mass

    Res = 1.
    r5 = r2*10.0
    while (Res > 1e-4):
        ur = u2/(r5/r2  -  1.)
        p5 = p2 +  r2*(ur + u2)*(ur + u2)*( 1. - r2/r5 )
        h5 = h2 + 0.5*(p5 - p2)*( 1./r2 + 1./r5 )

        Air_1.HP = h5,p5; Air_1.equilibrate('HP',loglevel=0,rtol=1e-4)
        Res      = np.abs(r5 - Air_1.density)/r5 
        r5       = Air_1.density

    s5 = Air_1.entropy_mass    
    u5 = (r2/r5)*(u2+ur) - ur
    t5 = Air_1.T
    s5 = Air_1.entropy_mass
    g5 = Air_1.cp_mole/Air_1.cv_mole
    Rg5= ct.gas_constant/Air_1.mean_molecular_weight
    a5 = np.sqrt(g5*(ct.gas_constant/Air_1.mean_molecular_weight)*t5)
    r5 = Air_1.density
    Mr = (ur + u2)/a2   
    mu5 = Air_1.viscosity #Pa.s
    tc5 = Air_1.thermal_conductivity #W/(m.K)
    return p5,t5,r5,g5,Rg5,a5,h5,s5,mu5,tc5,Air_1['N2'].X,Air_1['O2'].X,Air_1['NO'].X,Air_1['O'].X

#### The Equilibrium Condition

For some initial shock tube configurations, the reflected shock wave interacts successively with the gas interface (that separates driver and driven gases) and the shock tube end wall. These reflections become continuously weaker until an equilibrium condition is reached in the test gas. These phenomena increase the reflected temperature, pressure and enthalpy and this mode of operation is also known as *over tailor condition*. As these additional shock waves diminish in intensity, [4 and 5] assume these compressions can be approximated by multiple isentropic processes.

Accordingly to [6], the entropy change across a normal shock can be calculated for a calorically perfect gas as follows: 
$$ s_2 - s_1 = c_p ln\left[ \left(  1 + \frac{2\gamma}{\gamma+1}(M^2 -1) \right)
                            \left( \frac{2 + (\gamma -1)M^2}{(\gamma+1)M^2} \right)\right]
               -\Re ln \left[  1 + \frac{2\gamma}{\gamma+1}(M^2 -1) \right]  $$
where $c_p$ an $\Re$ are the specific heat and individual gas constant.

Using typical air values of c$_p$ = 1 kJ/(kg.K), $\Re$ = 0.287 kJ/(kg.K) and s$_1$ = 6.8 kJ/(kg.K), one can see the entropy increases 12% for M=5, a typical incident Mach Ms, and 3% for M=2.5, a typical reflected Mach number. The small increase in entropy in the latter case helps us to estimate the reflected or reservoir condition from measured pressures p$_{5E}$ by assuming and isentropic transformation from the calculated reflected state to experimental reflected state of measured pressure $p_{5E}$.    

Thus, the `Shock5E_Exp` function calculates the final reflected condition from reflected entropy $s_5$ and measured pressure $p_{5E}$ by assuming an isentropic evolution of equilibrium flows; the function also returns some reservoir parameters as enthalpy, viscosity and mole fraction.  

In this code, the `Shock5E_Exp` is always calculated; however, its output only diverges from `Shock25` output when an experimental measure of the reflected condition is given. 

-------------------------------------------
[4] Copper, J. A. (1962). <i>Experimental investigation of the equilibrium interface technique</i>. The Physics of Fluids, 5(7), 844-849.

[5] Minucci, M. A. S., Nagamatsu, H. T. (1993). <i>Hypersonic shock-tunnel testing at an equilibrium interface condition of 4100 K</i>. Journal of thermophysics and heat transfer, 7(2), 251-260.

[6] Anderson, J. D. (1990). Chapter: One-Dimensional Flow. <i>Modern compressible flow: with historical perspective.</i> Tata McGraw-Hill Education. pp 91.

In [5]:
def Shock5E_Exp(s5,p5E):
    Air_1.SP = s5,p5E
    Air_1.equilibrate('SP',loglevel=0) 
    
    t5E = Air_1.T #temperature
    h5E = Air_1.enthalpy_mass #specific enthalpy 
    s5E = Air_1.entropy_mass #specific entropy
    g5E = Air_1.cp_mole/Air_1.cv_mole #adiabatic index
    r5E = Air_1.density #density
    
    Rg5E = ct.gas_constant/Air_1.mean_molecular_weight #individual gas constant
    a5E  = np.sqrt(g5E*Rg5E*t5E) #sound speed
    
    mu5E = Air_1.viscosity # viscosity Pa.s
    tc5E = Air_1.thermal_conductivity#thermal conductivy  W/(m.K)
    return Rg5E,a5E,p5E,t5E,r5E,g5E,h5E,s5E,mu5E,tc5E,Air_1['N2'].X,Air_1['O2'].X,Air_1['NO'].X,Air_1['O'].X

#### Nozzle Throat Condition

When assuming an equilibrium flow, as is assumed in `Eq_At_M_Area` and `Equilibrium_Area` functions, the nozzle exit condition is calculated by an optimization process that compares calculated and input nozzle exit area ratios. Thus, the `ThroatM1` is used to provide density and speed at the nozzle throat in order to calculate the nozzle exit area ratio of equilibrium flows from the following relation (mass conservation):  
$$ \frac{A_{exit}}{A_{throat}} = \frac{r_{throat}. u_{throat}}{r_{exit} . u_{exit}} $$

By means of optimization, the nozzle throat parameters are calculated as the following iterative steps:

1. guess the nozzle throat pressure p$_t$ as the frozen isentropic pressure at sonic condition; 
2. calculate local enthalpy $h_t$ and sound speed ($a_t = \sqrt{ g_t.R_t.T_t }$) assuming an isentropic flow* from reservoir condition (here, Cantera is used)
3. calculate the local Mach from $M_t = \sqrt{ 2(h_{5E} - h_{t}) }/a_t $ (adiabatic assumption);
4. compare the calculated $M_t$ with the expected value at nozzle throat (= 1) and guess a new p$_t$;
5. repeat steps 2, 3 and 4 until convergence criteria is achieved. 

*The assumption of adiabatic and equilibrium chemically reacting nozzle flow guarantee an isentropic transformation, as can be seen in [1].

The optimization process is done by the `scipy.optimize.root` function to find the pressure $p_t$ that provides a flow at sonic conditions and thus specify the nozzle throat parameters.

The `ThroatM1` gets reservoir pressure, heat ratio, entropy and enthalpy as input and returns nozzle throat density and speed.  

---------------------

[1] Anderson Jr, J. D. (2006). Chapter: Inviscid High-Temperature Equilibrium Flows. <i>Hypersonic and high-temperature gas dynamics</i>. American Institute of Aeronautics and Astronautics. pp 618.

In [6]:
def ThroatM1(p5E,g5E,s5E,h5E):
    M_Match = 1.0
    Pis = p5E*(1. + .5*(g5E-1)*M_Match*M_Match)**(-g5E/(g5E-1)) #pressure guess

    def Throat_M(p):
        global G,Rg,R,U,T,R
        Air_1.SP = s5E,p
        Air_1.equilibrate('SP',loglevel=0,rtol=1e-4)

        H = Air_1.enthalpy_mass
        T = Air_1.T
        G = Air_1.cp_mole/Air_1.cv_mole
        R = Air_1.density
        Rg = ct.gas_constant/Air_1.mean_molecular_weight

        U = np.sqrt( 2.0*(h5E - H)  )
        M = U/np.sqrt( G*Rg*T )
        return M - M_Match

    pt = root(Throat_M, Pis).x
    return R,U

#### Adiabatic Frozen Nozzle Flow

The `Frozen_Area` function calculates the nozzle exit parameters from calculated reservoir state and nozzle area ratio $Ar$ assuming the frozen flow, a well known limiting case.  

First, the nozzle exit Mach is calculated from the the Mach number that satisfies the following expression in `Aux_M` (Area-Mach relation):
$$0 =  Ar.M - \left( \frac{g_{5E}+1}{2}\right)^{-\frac{g_{5E}+1}{2(g_{5E}-1)}}\left(1 + \frac{g_{5E}-1}{2}M^2\right)^{\frac{g_{5E}+1}{2(g_{5E}-1)}} $$
                                            
Then, the nozzle exit temperature, pressure and density is calculated from the ordinary frozen flow relations. In order to calculate the Pitot pressure, an iterative proccess similar to that applied in `Shock12` is used here to estimate the conditions created by the normal shock formed ahead an hypothetical Pitot probe. Thus, the nozzle flow is assumed to be frozen only from the reservoir condition to the region before the normal shock. 

The stagnation state is specified from the total pressure, calculated assuming an insentropic compression, and the reservoir enthalpy are used to specify the stagnation condition by assuming an equilibrium flow. 

The index '*1*' and *2* are used to indicate the free and post normal shock flows while '*to*' stands for stagnation condition.  

The function also calculates the nozzle exit:
   - shock stand-off distance ratio *dD=d/D* for a sphere of diameter *D* as proposed by [1]
    $$dD = \frac{d}{D} = 1 - \sqrt{1 - \left(\frac{u_2}{a_{to}}\right)^2} $$ 
   - gas composition 
   - fluorescence lifetime *tau1* of the first electronically excited state of Nitric Oxide *NO(A,v=0)* (see [2]), a parameter sensible to local density and chemical composition, and particularly useful for flow diagnostics based in molecular tagging techniques.  
   - characteristic fluorescence lenght *vtau1*: the flow velocity x fluorescence lifetime
   - viscosity
   - thermal conductivity
   - Reynolds number in units of 1/m
   - mass flow rate for a given area of interest *Am*, an input parameter
   

The `Frozen_Area` function assumes an adiabatic frozen nozzle flow to calculate the freestream condition; the function gets 
reservoir parameters (*5E in subscript*), nozzle exite area ratio (*Ar*) and area of interest (*Am*) as input and returns freestream parameters as freestream speed, Pitot pressure and mole fraction. 

-----------------------
[1] Shankara, T. S., & Sreekanth, A. K. (1977). Shock stand‐off distance for a sphere. Journal of Applied Physics, 48(4), 1765-1765.

[2] Paul, P. H., Gray, J. A., Durant Jr, J. L., & Thoman Jr, J. W. (1994). Collisional quenching corrections for laser-induced fluorescence measurements of NO A2Sigma (+). AIAA journal, 32(8), 1670-1675.

[3] Matos, P. A. D. S., Barreta, L. G., & Martins, C. A. (2018). Velocity and NO-lifetime measurements in an unseeded hypersonic air flow. Journal of Fluids Engineering, 140(12).

In [50]:
def Frozen_Area(p5E,t5E,r5E,g5E,Rg5E,h5E,Ar,Am):
    Aux_M = lambda M: Ar*M-((0.5*(g5E+1.))**(-0.5*(g5E+1)/(g5E-1)))*(1+0.5*(g5E-1)*M*M)**(0.5*(g5E+1)/(g5E-1))
    M = brenth(Aux_M,1.0,20.0)

    t1 = t5E/(1.0 + 0.5*(g5E-1)*M*M)
    p1 = p5E/((1.0 + 0.5*(g5E-1)*M*M)**(g5E/(g5E-1)))
    r1 = r5E/((1.0 + 0.5*(g5E-1)*M*M)**( 1./(g5E-1)))

    us = M*np.sqrt(g5E*Rg5E*t1)
    h1 = h5E - 0.5*us**2

    # post normal shock condition:
    r2 = r1*10; Res = 1
    while (Res > 1e-4):
        p2 = p1 +  r1*us*us*(1.0 - r1/r2        )
        h2 = h1 + 0.5*us*us*(1.0 - (r1/r2)**2.0 )
        
        Air_1.HP = h2,p2
        Air_1.equilibrate('HP',loglevel=0,rtol=1e-4);

        Res = np.abs(r2 - Air_1.density)/r2 
        r2  = Air_1.density;
        u2  = us*(r1/r2)
           
    g2  =  Air_1.cp_mole/Air_1.cv_mole;  
    Rg2 =  ct.gas_constant/Air_1.mean_molecular_weight ; 
    T2  =  Air_1.T     
    M2  =  u2/np.sqrt(g2*Rg2*T2)
    a2  =  np.sqrt(g2*Rg2*T2)
    
    #at the stagnation point:
    pto = p2*(1 + 0.5*(g2-1.)*M2**2. )**(g2/(g2-1))
    Air_1.HP = h5E,pto; Air_1.equilibrate('TP',loglevel=0,rtol=1e-4);
    gto  =  Air_1.cp_mole/Air_1.cv_mole;  
    Rto =  ct.gas_constant/Air_1.mean_molecular_weight ; 
    Tto  =  Air_1.T     
    ato  =  np.sqrt(gto*Rto*Tto)

    dD = 1 - (1 - (u2/ato)**2)**0.5

    
    Air_1.TP = t5E,p5E
    Air_1.equilibrate('TP',loglevel=0,rtol=1e-4);

    X = np.array([Air_1['N2'].X, Air_1['H2O'].X, Air_1['CO2'].X,  Air_1['CO'].X, \
                  Air_1['O2'].X,  Air_1['OH'].X,   Air_1['H'].X,   Air_1['O'].X, \
                  Air_1['H2'].X,  Air_1['NO'].X,  Air_1['Ar'].X, Air_1['N2O'].X, \
                  Air_1['NO2'].X]).T

    tau1 = tfl(p1,t1,X)
    vtau1 = us*tau1*1e-9

    Air_1.TP = t1,p1
    mu1  = Air_1.viscosity; #Pa.s
    tc1  = Air_1.thermal_conductivity; #W/(m.K)
    Rel1 = r1*us/mu1
    mp1  = r1*us*Am 

    return Ar,p1/1e3,t1,1e3*r1,us,M,1e-3*pto,tau1,1e6*vtau1,1e6*mu1,1e3*mp1,1e-5*Rel1,1e3*tc1,1e2*dD,\
           1e2*Air_1['N2'].X,1e2*Air_1['O2'].X,1e2*Air_1['NO'].X,1e2*Air_1['O'].X

#### Adiabatic Equilibrium Nozzle Flow

This function, `Equilibrium_Area`, calculates the nozzle exit condition whose calaculated area ratio matches the input area ratio; the calculation assumes an equilibrium flow from nozzle reservoir to nozzle exit.  

From reservoir entropy and a guessed nozzle exit pressure, the `Aux_Ar` function specifies the freestream paramaters by assuming an equilibrium flow. Then, an area ratio *Ac* is calculated from freestream data as shown in the <b>Nozzle Throat Condition</b> discussion. In the `Equilibrium_Area` function, the `scipy.optmization.brenth` optimization function is used to find the nozzle exit pressure that satisfies:
$$ Ac - Ar = 0 ,$$
where *Ac* and *Ar* are the calculated and input area ratio. This procedure is again performed in the next `Eq_At_M_Area` block function.  

Once the freestream is specified, all the procedure to calculate the stagnation condition is the same as explained in <b>Adiabatic Frozen Nozzle Flow</b>.

The `Equilibrium_Area` function assumes an chemical equilibrium nozzle flow to calculate the freestream condition; the function gets reservoir parameters (*5E in subscript*), nozzle exite area ratio (*Ar*) and area of interest (*Am*) as input, and returns freestream parameters as freestream speed, Pitot pressure and mole fraction. 

In [51]:
def Equilibrium_Area(p5E,t5E,r5E,g5E,h5E,s5E,Ar,Am):
    Aux_M = lambda M: Ar*M - ((0.5*(g5E+1))**(-0.5*(g5E+1)/(g5E-1)))*(1 + 0.5*(g5E-1)*M*M)**(0.5*(g5E+1)/(g5E-1))    
    Mis   = brenth(Aux_M,1.0,20.0)
    pis   = p5E/((1.0 + 0.5*(g5E-1)*Mis*Mis)**(g5E/(g5E-1))) 

    Rz,Uz = ThroatM1(p5E,g5E,s5E,h5E)
    def Aux_Ar(p):
        global M,R,T,H,U,Ac
        Air_1.SP = s5E,p; Air_1.equilibrate('SP',loglevel=0,rtol=1e-4)
        H  = Air_1.enthalpy_mass
        T  = Air_1.T
        G  = Air_1.cp_mole/Air_1.cv_mole
        R  = Air_1.density
        Rg = ct.gas_constant/Air_1.mean_molecular_weight

        U = np.sqrt( 2.0*(h5E - H)  )
        M = U/np.sqrt( G*Rg*T )
            
        Ac = Rz*Uz/(R*U)
        return Ac - Ar
    
    
    p1          = brenth(Aux_Ar, pis*0.5, pis*2)
    r1,t1,h1,us = R,T,H,U

    r2 = r1*10; Res = 1.0
    while (Res>1e-4):
        p2 = p1 +  r1*us*us*( 1.0 - r1/r2        )
        h2 = h1 + 0.5*us*us*( 1.0 - (r1/r2)**2.0 )

        Air_1.HP = h2,p2; Air_1.equilibrate('HP',loglevel=0,rtol=1e-4)
        Res      = np.abs(r2 - Air_1.density)/r2
        r2       = Air_1.density
        u2       = us*(r1/r2)

    g2  = Air_1.cp_mole/Air_1.cv_mole
    Rg2 = ct.gas_constant/Air_1.mean_molecular_weight
    T2  = Air_1.T
    M2  = u2/np.sqrt(g2*Rg2*T2)

    #at the stagnation point:
    pto = p2*(1 + 0.5*(g2-1.)*M2**2. )**(g2/(g2-1))
    Air_1.HP = h5E,pto; Air_1.equilibrate('HP',loglevel=0,rtol=1e-4);
    gto  =  Air_1.cp_mole/Air_1.cv_mole;  
    Rto  =  ct.gas_constant/Air_1.mean_molecular_weight ; 
    Tto  =  Air_1.T     
    ato  =  np.sqrt(gto*Rto*Tto)

    dD = 1 - (1 - (u2/ato)**2)**0.5
    
    Air_1.TP = t1,p1; Air_1.equilibrate('TP',loglevel=0,rtol=1e-4)   
    X = np.array([Air_1['N2'].X, Air_1['H2O'].X,  Air_1['CO2'].X,  Air_1['CO'].X, \
                  Air_1['O2'].X,  Air_1['OH'].X,    Air_1['H'].X,   Air_1['O'].X, \
                  Air_1['H2'].X,  Air_1['NO'].X,   Air_1['Ar'].X, Air_1['N2O'].X, \
                  Air_1['NO2'].X]).T

  
    tau1 = tfl(p1,t1,X)
    vtau1 = us*tau1*1e-9
    
    mu1  = Air_1.viscosity #Pa.s
    tc1  = Air_1.thermal_conductivity #W/(m.K)
    Rel1 = r1*us/mu1
    mp1  = r1*us*Am 

    return Ar,p1/1e3,t1,1e3*r1,us,M,1e-3*pto,tau1,1e6*vtau1,1e6*mu1,1e3*mp1,1e-5*Rel1,1e3*tc1,1e2*dD,\
           1e2*Air_1['N2'].X,1e2*Air_1['O2'].X,1e2*Air_1['NO'].X,1e2*Air_1['O'].X

####  Instantaneous transition from Equilibrium to Frozen Flow

This function, `Eq_At_M_Area`, divides the nozzle into two regions of instantaneous transition. The first region is assumed to be an equilibrium flow and lies from nozzle entrance to the position where the Mach number matches a Mach M_Match given by the user as input data; the second region, where M > M_Match, is assumed to be a frozen flow.

First, the `At_M_Match(p)` function is used by the `root()` optimization routine to identify the pressure at the condition that the local Mach number matches the M_Match value. Thus, the equilibrium flow can be localy specified at M=M_Match. 

Second, and analogously as before, `At_Nozzle_Exit(p)` function is used to calculate the nozzle exit/freestream pressure, by knowing the nozzle entrance entropy and keeping the composition found in the above procedure, that provides a area ratio that matches the one specified by the user.   

Once the freestream is specified, the stagnation condition is calculated as explained in <b>Adiabatic Frozen Nozzle Flow</b>.

<img src="Nozzle_Jupy.png" width=600 height=600 />

The `Eq_At_M_Area` function gets reservoir parameters (*5E in subscript*), nozzle exite area ratio (*Ar*), area of interest (*Am*) and *M_Match* (the Mach number to be matched) as input, and returns freestream parameters as freestream speed, Pitot pressure and mole fraction. 



In [52]:
def Eq_At_M_Area(Ar,g5E,p5E,h5E,t5E,s5E,r5E,M_Match,Am):                
    pis = lambda M: p5E*(1. + .5*(g5E-1)*M*M)**(-g5E/(g5E-1))
    
    def At_M_Match(p):
        global G,Rg,R,U,T,R
        Air_1.SP = s5E,p; Air_1.equilibrate('SP',loglevel=0,rtol=1e-4)

        H  = Air_1.enthalpy_mass
        T  = Air_1.T
        G  = Air_1.cp_mole/Air_1.cv_mole
        R  = Air_1.density
        Rg = ct.gas_constant/Air_1.mean_molecular_weight

        U = np.sqrt( 2.0*(h5E - H)  )
        M = U/np.sqrt( G*Rg*T )
        return M - M_Match
 
    P        = root(At_M_Match, pis(M_Match)).x
    Air_1.SP = s5E,P; Air_1.equilibrate('SP',loglevel=0,rtol=1e-4)
    gM       = G
    RgM      = Rg
     
    Rz,Uz    = ThroatM1(p5E,g5E,s5E,h5E)    
    Ar_atM   = Rz*Uz/(R*U)
    if (Ar < Ar_atM):
        print('ERROR: Ar_AtM > Ar\nArea ratio at Mach=M_Match must be less or equal to nozzle-exit area ratio')
    
    def At_Nozzle_Exit(p):
        global h1,g1,Rg1,r1,us,t1,r1
        Air_1.SP = s5E,p
        
        h1  = Air_1.enthalpy_mass
        t1  = Air_1.T
        g1  = Air_1.cp_mole/Air_1.cv_mole
        r1  = Air_1.density
        Rg1 = ct.gas_constant/Air_1.mean_molecular_weight

        us = np.sqrt( 2.0*(h5E - h1)  )
        M1 = U/np.sqrt( g1*Rg1*t1)
        
        Ar_Exit = Rz*Uz/(r1*us)
        
        return Ar_Exit - Ar
    
    p1 = brenth( At_Nozzle_Exit, pis(1), pis(10.0))
    
    r2 = r1*10
    Res = 1.0
    while (Res > 1.e-4):
        p2 = p1 +  r1*us*us*( 1.0 - r1/r2        )
        h2 = h1 + 0.5*us*us*( 1.0 - (r1/r2)**2.0 )

        Air_1.HP = h2,p2; Air_1.equilibrate('HP',loglevel=0,rtol=1e-4)

        Res = np.abs(r2 - Air_1.density)/r2
        r2  = Air_1.density

    u2 = us*(r1/r2)

    g2  = Air_1.cp_mole/Air_1.cv_mole
    Rg2 = ct.gas_constant/Air_1.mean_molecular_weight
    T2  = Air_1.T
    M2  = u2/np.sqrt(g2*Rg2*T2)

    #at the stagnation point:
    pto = p2*(1 + 0.5*(g2-1.)*M2**2. )**(g2/(g2-1))
    Air_1.HP = h5E,pto; Air_1.equilibrate('TP',loglevel=0,rtol=1e-4);
    gto  =  Air_1.cp_mole/Air_1.cv_mole;  
    Rto =  ct.gas_constant/Air_1.mean_molecular_weight ; 
    Tto  =  Air_1.T     
    ato  =  np.sqrt(gto*Rto*Tto)

    dD = 1 - (1 - (u2/ato)**2)**0.5
    
    Air_1.TP = T,P
    Air_1.equilibrate('TP',loglevel=0)

    X = np.array([Air_1['N2'].X, Air_1['H2O'].X,   Air_1['CO2'].X,  Air_1['CO'].X, \
                  Air_1['O2'].X,  Air_1['OH'].X,     Air_1['H'].X,   Air_1['O'].X, \
                  Air_1['H2'].X,  Air_1['NO'].X,    Air_1['Ar'].X, Air_1['N2O'].X, \
                  Air_1['NO2'].X]).T

    tau1 = tfl(p1,t1,X)

    vtau1 = us*tau1*1e-9
    
    
    Air_1.TP = t1,p1
  
    mu1  = Air_1.viscosity #Pa.s
    tc1  = Air_1.thermal_conductivity #W/(m.K)
    Rel1 = r1*us/mu1
    mp1  = r1*us*Am 

    return Ar,p1/1e3,t1,1e3*r1,us,M,1e-3*pto,tau1,1e6*vtau1,1e6*mu1,1e3*mp1,1e-5*Rel1,1e3*tc1,1e2*dD,\
           1e2*Air_1['N2'].X,1e2*Air_1['O2'].X,1e2*Air_1['NO'].X,1e2*Air_1['O'].X

#### Shock Tunnel Calculation
This block of code maps the shock tunnel for different input parameters following the procedures explained in the previous blocks. The `Input` function creates the `C5` array to condense shocked, reflected, and reservoir shock tube data and the three arrays Fr_Exit, Eq_Exit, and EF_Exit to be filled with standardized `Frozen_Area`, `Equilibrium_Area`, and `Eq_At_M_Area` nozzle exit outputs. 

To run the code, driver, driven, and nozzle data must be defined by the user in the interface control (based on ipywidgets library) at the end of the following code. The input data are:

**Driver Input tab:**
   - Pressure *p4*: all the shock tunnel data are calculated for a fixed driver pressure value;
   - Temperature *T4*: all the calculation considers a fixed value for driver temperature;
   - Gas: when clicked, a dropdown list shows a list of some commom gases. For a different gas, one can select *Other*\* in the dropped list and both gas *heat capacity ratio* and *mean molecular weight* (in unit of atomic mass) must be comma separated entered in *Other*\* field. For example, insert *1.66, 32* for oxygen gas. This data will be used to calculate *Ms* by `Calc_Ms`.
   
**Driven Input tab:**
   - Cantera Reaction: as default, the Gri3.0 gas mechanism is used to calculate the air parameters [1]. Other mechanisms can be found in [2];
   - Composition: different gas compositions can be inserted in this field; for example: *N2:0.5, NO:0.25, CO2:0.25*. An approximated air composition is used as default;
   - Pressure: the shock tube is calculated for different driven pressure values. In this field, a range of values must be inserted in the "*initial pressure, final pressure, number of steps*" form;  
   - Temperature *T1*: all the calculation considers a fixed value for driven temperature.

**Reservoir Input tab:**
   - In the *'p5 Data'* radio button, two options are shown: *'Calculated p5'* and *'Measured p5e'*.
   
     If the first option is chosen, the reservoir pressure is assumed to be the same as the calculated *p5*.
     
     If *'Measured p5e'* is chosen, the user must insert a list of experimental driven pressure in the *'p1 Data'* field and its respective measured reservoir pressure in the *'p5e Data'* field. This experimental data is fitted with a polynomial fit that will be used to estimate the reservoir pressure for any given *p1*; the poly fit degree must be inserted in the *'Poly fit Degree'* field.      
   
**Nozle Exit Input tab:** 
   - Area Ratio: the freestream parameter is calculated for a range of *p1* and nozzle exit area ratio *Ar*. In this field, a range of values must be inserted in the "*initial area ratio, final area ratio, number of steps*" form;  
   - Equilibrium to Frozen at *M*: this is the Mach number that will me used in the `Eq_At_M_Area` to separate equilibrium from frozen flow; 
   - Area of interest: area, in units 
      
      
Once all input data is inserted, the user must click on the *Plot* button.     
    
------------------
[1] [Gri-Mech, an overview](http://combustion.berkeley.edu/gri-mech/overview.html)

[2] [Cantera: Input Files](https://cantera.org/tutorials/input-files.html)  

In [53]:
style = {'description_width': 'initial'}

p4    = widgets.FloatText(description="p4 (MPa):", value=20.0, continuous_update=False)
T4    = widgets.FloatText(description="T4 (K):"  , value=298, continuous_update=False)
Gas   = widgets.Dropdown(description=r'Gas:',options=[('He',[1.6666,4]), ('air',[1.4,28.966]),('N2',[1.4,28.0134]),('H2',[1.41,2.016]),('Ar',[1.666,39.949]),('CO2',[1.3,44]),('Other*',[0,0])], continuous_update=False)
Other = widgets.Text(description='Other*',placeholder='gamma, molecular weight', continuous_update=True)

iD1 = widgets.HBox([ p4,T4])
iD2 = widgets.HBox([ Gas,Other])
UI_Driver = widgets.VBox([iD1,iD2])

Mech = widgets.Text(description='Cantera Reaction:',value='gri30_highT.cti', continuous_update=False,style=style)
Comp = widgets.Text(description='Composition:',value="N2: 0.7808, O2:0.209", continuous_update=False)
p1   = widgets.Text(description='p1 (kPa):',value='10,100,5',placeholder='start, stop, n of intervals', continuous_update=False)
T1   = widgets.FloatText(description="T1 (K):"  , value=300, continuous_update=False)

id1 = widgets.HBox([Mech])
id2 = widgets.HBox([Comp])
id3 = widgets.HBox([p1,T1])
UI_Driven = widgets.VBox([id1,id2,id3])


p5d  = widgets.RadioButtons( options=['Calculated p5', 'Measured p5e'], value='Measured p5e', description='p5 Data:',disabled=False)
deg  = widgets.FloatText(description="Poly fit Degree:", value=4, placeholder = 'Insert an integer',continuous_update=False,style=style)
p1m = widgets.Textarea(description='p1 Data (kPa):',value='10,25,50,75,100,125',placeholder=r'p1$_1$, p1$_2$, ..., p1$_n$', continuous_update=False,style=style)
p5m = widgets.Textarea(description='p5e Data (MPa):',value='12.49,12.53,12.86,13.29,13.81,14.39',placeholder=r'p5$_1$, p5$_2$, ..., p5$_n$', continuous_update=False,style=style)

ir1 = widgets.HBox([p5d,deg])
ir2 = widgets.HBox([p1m,p5m])
UI_Reserv = widgets.VBox([ir1,ir2])

#Exit
AR   = widgets.Text(description='Area Ratio:',value='10,600,5',placeholder='start, stop, n of intervals', continuous_update=False)
EF   = widgets.FloatText(description="Equilibrium to Frozen at M:"  , value=1.0, continuous_update=False,style=style)
ARm  = widgets.Text(description=r'Area of Interest (cm$^2$):',value='1.0', continuous_update=False,style=style)

ie1 = widgets.HBox([AR,ARm])
ie2 = widgets.HBox([EF])


UI_Exit = widgets.VBox([ie1,ie2])
Run = widgets.ToggleButtons(options=['Stoped','Run'],disabled=False,button_style='success')
Bar = widgets.FloatProgress(value=10, min=0, max=10, step=1, description='Press Run', bar_style='success', 
      orientation='horizontal')

UI_Run = widgets.HBox([Run])
def Input(p4,t4,Gas,Other,Mech,Comp,p10,t1,p5d,deg,p1m,p5m,AR,ARm,M_Match,Run2):
    global C5,Fr_Exit,Eq_Exit,EF_Exit,NE_AR,M_EF,Air_1
    
    Air_1   = ct.Solution(Mech)
    Air_1.X = Comp
    Air_1.transport_model = 'Multi'
    
    p1_1 = p10.split(',')  
    AR_1 = AR.split(',')
    Gas_1 = [Gas[0],Gas[1]]
    M_EF = M_Match

    Bar.max = float(p1_1[2])*float(AR_1[2])
    Bar.value = float(p1_1[2])*float(AR_1[2]) 
    if Run2=='Run': 
        Bar.bar_style='danger'
        Bar.description='<i>Running</i>'
        
        P1    = np.linspace(  float(p1_1[0]), float(p1_1[1]), float(p1_1[2]))*1e3
        NE_AR = np.linspace(  float(AR_1[0]), float(AR_1[1]), float(AR_1[2])) 

        Fr_Exit = np.zeros((P1.size, NE_AR.size, 18))
        Eq_Exit = np.zeros((P1.size, NE_AR.size, 18))
        EF_Exit = np.zeros((P1.size, NE_AR.size, 18))
        C5      = np.zeros((P1.size,             42))

        if Gas[0]==0:
            print('Insert driver gas gamma and mean mecular weight')
            Other_1 = Other.split(',')
            g4  = float(Other_1[0]); mw4 = float(Other_1[1]) 
            print(g4,mw4)
        else:
            g4  = Gas_1[0]; mw4 = Gas_1[1]
        a4 = np.sqrt( g4*(8314.5/mw4)*t4 )    
                
        if p5d =='Measured p5e':
            p1_m = np.array(p1m.split(','), dtype=np.float32)
            p1_x = np.linspace(p1_m[0],p1_m[-1],20)
            p5_m = np.array(p5m.split(','), dtype=np.float32)           
            f = np.poly1d(np.polyfit(p1_m*1e3, p5_m*1e6, deg))   
            
            plt.figure('p1xp5',figsize=(9,4))
            plt.plot(p1_m,p5_m,'ko',label='Experimental Data')
            plt.plot(p1_x,f(p1_x*1e3)/1e6,'r-',label='Fit Data')
            plt.grid()
            plt.xlabel('p1 (kPa)',fontsize=14)
            plt.ylabel('p5 (MPa)',fontsize=14)
            plt.legend()
            plt.show()

        cont = 0.
        for i,p1 in enumerate(P1):
            Air_1.TP = t1,p1;
            Air_1.equilibrate('TP',rtol=1e-4)
            g1 = Air_1.cp_mole/Air_1.cv_mole
            a1 = np.sqrt( g1*(ct.gas_constant/Air_1.mean_molecular_weight)*t1 )

            Ms = brenth(Calc_Ms, 1.01, 10, args=(p1,a1,g1,g4,a4,1e6*p4))
            us = Ms*a1 
            
            u2,p2,t2,r2,g2,a2,h2,s2,mu2,tc2,xN2_2,xO2_2,xNO_2,xO_2   =  Shock12(t1,p1,us)
            p5,t5,r5,g5,Rg5,a5,h5,s5,mu5,tc5,xN2_5,xO2_5,xNO_5,xO_5  =  Shock25(t2,p2,r2,u2,a2)

            if p5d =='Measured p5e':
                p5E = f(p1)

            if p5d =='Calculated p5':
                p5E = p5      

            Rg5E,a5E,p5E,t5E,r5E,g5E,h5E,s5E,mu5E,tc5E,xN2_5E,xO2_5E,xNO_5E,xO_5E = Shock5E_Exp(s5,p5E)

            C5[i,:] = us,Ms,p1/1e3,\
                      p2/1e3,t2,r2,g2,a2,h2/1e6,s2/1e3,1e6*mu2,1e3*tc2,xN2_2,xO2_2,xNO_2,xO_2,\
                      p5/1e6,t5,r5,g5,a5,h5/1e6,s5/1e3,1e6*mu5,1e3*tc5,xN2_5,xO2_5,xNO_5,xO_5,\
                      p5E/1e6,t5E,r5E,g5E,a5E,h5E/1e6,s5E/1e3,1e6*mu5E,1e3*tc5E,xN2_5E,xO2_5E,xNO_5E,xO_5E

            for j,Aratio in enumerate(NE_AR):
                Fr_Exit[i,j,:] = Frozen_Area(p5E,t5E,r5E,g5E,Rg5E,h5E,Aratio,float(ARm)/1e4)
                Eq_Exit[i,j,:] = Equilibrium_Area(p5E,t5E,r5E,g5E,h5E,s5E,Aratio,float(ARm)/1e4)
                EF_Exit[i,j,:] = Eq_At_M_Area(Aratio,g5E,p5E,h5E,t5E,s5E,r5E,M_Match,float(ARm)/1e4)
                cont = cont + 1
                Bar.value = cont
      
        Bar.value = Bar.max   
        Bar.bar_style='success' 
        Bar.description='Finished'; time.sleep(0.5); Bar.description='Press Run'
    #Run.value = 'Stoped'
        
    return
Inp = widgets.interactive_output(Input,{  'p4':p4,    't4':T4,       'Gas':Gas, 'Other':Other,
                                        'Mech':Mech,'Comp':Comp,     'p10':p1,     't1':T1,
                                         'p5d':p5d,  'deg':deg,      'p1m':p1m,   'p5m':p5m,
                                          'AR':AR,   'ARm':ARm,  'M_Match':EF,   'Run2':Run    })
UI_Tab = widgets.Tab()
UI_Tab.children = [UI_Driver, UI_Driven, UI_Reserv , UI_Exit]
UI_Tab.set_title(0, 'Driver Input')
UI_Tab.set_title(1, 'Driven Input')
UI_Tab.set_title(2, 'Reservoir Input')
UI_Tab.set_title(3, 'Nozzle Exit Input')

Title = widgets.HTML(value="<strong>SHOCK TUNNEL CALCULATION:</strong>")
UI = widgets.VBox([Title,UI_Tab,Run,Bar])
display(UI, Inp)


Output()

#### Shock Tube Results

Here the calculated shock tube parameters as driven pressure or reservoir enthalpy are shown in three different plots. To do that, the user must insert y and x axis attributes of each plot in the dropdown lists in the following UI.

In [54]:
style = {'description_width': 'initial'}
list_of_params = [('Incident Shock Speed, us',0),('Incident Mach Number, Ms',1), ('Pressure, p_1',2),
                  ('Pressure, p_2',3),('Temperature, T_2',4),('Density, r_2',5), ('Heat Capacity Ratio, gamma_2',6),
                  ('Sound Speed, a_2',7),('Specific Enthalpy, h_2',8),('Specific Entropy, s_2',9),('Viscosity, mu_2',10),
                  ('Thermal Conductivity, k_2',11),('N2, XN2_2', 12),('O2, XO2_2', 13),('NO, XNO_2', 14),
                  ('O, XO_2', 15),('Pressure, p_5',16),('Temperature, T_5',17),('Density, r_5',18),('Heat Capacity Ratio, gamma_5',19),
                  ('Sound Speed, a_5',20),('Specific Enthalpy, h_5',21),('Specific Entropy, s_5',22),('Viscosity, mu_5',23),
                  ('Thermal Conductivity, k_5',24),('N2, XN2_5', 25),('O2, XO2_5', 26),('NO, XNO_5', 27),('O, XO_5', 28),                  
                  ('Pressure, p_5e',29),('Temperature, T_5e',30),('Density, r_5e',31),('Heat Capacity Ratio, gamma_5e',32),
                  ('Sound Speed, a_5e',33),('Specific Enthalpy, h_5e',34),('Specific Entropy, s_5e',35),('Viscosity, mu_5e',36),
                  ('Thermal Conductivity, k_5e',37),('N2, XN2_5e',38),('O2, XO2_5e',39),('NO, XNO_5e', 40),('O, XO_5e', 41)]
                  
label = ['Incident Shock Speed (m/s)','Incident Mach' ,'Driven Pressure (kPa)', 
         'Shocked   Pressure (kPa)', 'Shocked   Temperature (K)', 'Shocked   Density '+r'(kg.m$^{-3}$)', 'Shocked   '+r'$\gamma$', 'Shocked   Sound Speed (m/s)', 'Shocked   Spec. Enthalpy '+r'(MJ.kg$^{-1}$)', 'Shocked   Spec. Entropy '+r'(kJ.kg$^{-1}$.K$^{-1}$)' , 'Shocked   Viscosity'+r' ($\mu$Pa.s)', 'Shocked   Thermal Conductivity'+r'W.m$^{-1}$.K$^{-1}$', 'Shocked   '+r'$X_{N_{2}}$', 'Shocked   '+r'$X_{O_{2}}$', 'Shocked   '+r'$X_{NO}$', 'Shocked   '+r'$X_{O}$',
         'Reflected Pressure (MPa)', 'Reflected Temperature (K)', 'Reflected Density '+r'(kg.m$^{-3}$)', 'Reflected '+r'$\gamma$', 'Reflected Sound Speed (m/s)', 'Reflected Spec. Enthalpy '+r'(MJ.kg$^{-1}$)', 'Reflected Spec. Entropy '+r'(kJ.kg$^{-1}$.K$^{-1}$)' , 'Reflected Viscosity'+r' ($\mu$Pa.s)', 'Reflected Thermal Conductivity'+r'W.m$^{-1}$.K$^{-1}$', 'Reflected '+r'$X_{N_{2}}$', 'Reflected '+r'$X_{O_{2}}$', 'Reflected '+r'$X_{NO}$', 'Reflected '+r'$X_{O}$',
         'Reservoir Pressure (MPa)', 'Reservoir Temperature (K)', 'Reservoir Density '+r'(kg.m$^{-3}$)', 'Reservoir '+r'$\gamma$', 'Reservoir Sound Speed (m/s)', 'Reservoir Spec. Enthalpy '+r'(MJ.kg$^{-1}$)', 'Reservoir Spec. Entropy '+r'(kJ.kg$^{-1}$.K$^{-1}$)' , 'Reservoir Viscosity'+r' ($\mu$Pa.s)', 'Reservoir Thermal Conductivity'+r'W.m$^{-1}$.K$^{-1}$', 'Reservoir '+r'$X_{N_{2}}$', 'Reservoir '+r'$X_{O_{2}}$', 'Reservoir '+r'$X_{NO}$', 'Reservoir '+r'$X_{O}$']

t = [];x = [];y = [];iD = [];val = [16,17,30]
for i in np.arange(3):
    t.append(widgets.HTML(value="<b>Plot "+str(i+1)+"</b>"))    
    x.append(widgets.Dropdown(description="x axis data:",value=2,options=list_of_params, continuous_update=False,style=style))
    y.append(widgets.Dropdown(description="y axis data:",value=val[i],options=list_of_params, continuous_update=False,style=style))
    iD.append(   widgets.VBox( [t[i],x[i],y[i]], layout=widgets.Layout( width='100%') )        )

UI_Plot = widgets.HBox( iD  )
Run = widgets.ToggleButtons(options=['Hide','Plot'],disabled=False,button_style='warning')
UI_Run = widgets.HBox([Run])


def Input(x0,x1,x2,y0,y1,y2,Run2):
    if Run2=='Hide':
        plt.clf()
        
    if Run2=='Plot': 
        plt.figure('Plots',figsize=[16,4])
        plt.subplot(1,3,1); plt.plot(C5[:,x0],C5[:,y0],'ko-')
        plt.xlabel(label[x0],fontsize=14); plt.ylabel(label[y0],fontsize=12);plt.grid()

        plt.subplot(1,3,2); plt.plot(C5[:,int(x1)],C5[:,int(y1)],'ko-')
        plt.xlabel(label[x1],fontsize=14); plt.ylabel(label[y1],fontsize=12);plt.grid()

        plt.subplot(1,3,3); plt.plot(C5[:,int(x2)],C5[:,int(y2)],'ko-')
        plt.xlabel(label[x2],fontsize=14); plt.ylabel(label[y2],fontsize=12);plt.grid()

        plt.tight_layout()
        plt.show()                   
    return
Title = widgets.HTML(value="<strong>DATA PLOTTING</strong>")
Inp = widgets.interactive_output(Input,{ 'x0':x[0],'x1':x[1],'x2':x[2], 'y0':y[0],'y1':y[1],'y2':y[2], 'Run2':Run                     })
UI = widgets.VBox([Title,UI_Plot,UI_Run])
display(UI,Inp)


Output()

#### Nozzle Exit Results

The calculated nozzle exit parameters are shown here through three maps created from frozen, equilibrium to frozen, and equilibrium flow calculations. All the three maps are plotted from the same attribute and the input data are as follows:

**Map Data:**
   - Y Axis: here, the user must select the y axis parameter from the dropdown list that includes all caculated shock tube parameters; the x axis is fixed and represents the nozzle exit throat-to-exit area ratio; 
   - Attribute: represents the freestream parameter to be plotted; the user must select the nozzle exit attribute to be plotted from the dropdown list in this field.
  
**Parameters:**<br>
 - Show Contour Levels: check it if you want to see linearly spaced contour levels on the plot;
 - Show Contour Levels 2: check it if you want to see logarithmically spaced contour levels on the plot;
 - Show Bars: check it to see a color bar of reference;
 - Number of Intervals: the number of countour levels to be plotted in the map;
 - Colormap: select a colormap from a list of option in the combobox list;
 - Shading: select the map shading type. 
 
Then, click on Plot button. 

In [58]:
style = {'description_width': 'initial'}

list_2 = [('Incident Shock Speed, us',0), ('Incident Mach, Ms', 1),('Pressure, p_1',2),
          ('Pressure, p_2',3),('Temperature, T_2',4), ('Density, r_2',5),('Heat Capacity Ratio, gamma_2',6),
          ('Sound Speed, a_2',7),('Specific Enthalpy, h_2',8),('Specific Entropy, s_2',9),('Viscosity, mu_2',10),
          ('Thermal Conductivity, k_2',11),('N2, XN2_2', 12),('O2, XO2_2',13),('NO, XNO_2', 14),('O, XO_2', 15),                 
          ('Pressure, p_5',16),('Temperature, T_5',17),('Density, r_5',18),('Heat Capacity Ratio, gamma_5',19),
          ('Sound Speed, a_5',20),('Specific Enthalpy, h_5',21),('Specific Entropy, s_5',22),('Viscosity, mu_5',23),
          ('Thermal Conductivity, k_5',24),('N2, XN2_5', 25),('O2, XO2_5',26),('NO, XNO_5', 27),('O, XO_5', 28),                  
          ('Pressure, p_5e',29),('Temperature, T_5e',30),('Density, r_5e',31), ('Heat Capacity Ratio, gamma_5e',32),
          ('Sound Speed, a_5e',33),('Specific Enthalpy, h_5e',34),('Specific Entropy, s_5e',35),('Viscosity, mu_5e',36),
          ('Thermal Conductivity, k_5e',37),('N2, XN2_5e',38),('O2, XO2_5e',39),('NO, XNO_5e',40),('O, XO_5e', 41)]


list_1 = [('Area Ratio, A/A*',0),('Pressure, p_e',1),('Temperature, T_e',2),
          ('Density, r_e',3),('Velocity, u_e',4),('Mach, M_e',5),
          ('Pitot Pressure, Pto_e',6),('NO(A) Fluorescence Lifetime, tau_e',7),('Characteristic Lenght, v_e.tau_e',8),
          ('Viscosity, mu_e',9),('Mass Flow Rate, mp_e',10), ('Reynolds/m, Re_e',11),
          ('Thermal Conductivity, tc_e',12),('Stand-off Distance, dD_e',13),('N2, XN2_e',14),
          ('O2, XO2_e',15),('NO, XNO_e',16),('O, XO_e',17)]


list_of_cmap = ['viridis', 'plasma', 'inferno', 'magma', 'cividis','Greys',
'Purples', 'Blues', 'Greens', 'Oranges', 'Reds','YlOrBr', 'YlOrRd', 'OrRd',
'PuRd', 'RdPu', 'BuPu','GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn','binary',
'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink','spring', 'summer', 'autumn', 'winter',
'cool', 'Wistia','hot', 'afmhot', 'gist_heat', 'copper','PiYG', 'PRGn', 'BrBG', 'PuOr',
'RdGy', 'RdBu','RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic','twilight',
'twilight_shifted', 'hsv','Pastel1', 'Pastel2', 'Paired', 'Accent','Dark2', 'Set1', 'Set2',
'Set3', 'tab10', 'tab20', 'tab20b', 'tab20c','flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg','gist_rainbow', 'rainbow', 'jet',
'nipy_spectral', 'gist_ncar']

list_of_shading = ['flat', 'nearest', 'gouraud', 'auto']

label_1 = ['Incident Shock Speed (m/s)','Incident Mach' ,'Driven Pressure (kPa)', 
           'Shocked   Pressure (kPa)', 'Shocked   Temperature (K)', 'Shocked   Density '+r'(kg.m$^{-3}$)', 'Shocked   '+r'$\gamma$', 'Shocked   Sound Speed (m/s)', 'Shocked   Spec. Enthalpy '+r'(MJ.kg$^{-1}$)', 'Shocked   Spec. Entropy '+r'(kJ.kg$^{-1}$.K$^{-1}$)' , 'Shocked   Viscosity'+r' ($\mu$Pa.s)', 'Shocked   Thermal Conductivity'+r'W.m$^{-1}$.K$^{-1}$', 'Shocked   '+r'$X_{N_{2}}$', 'Shocked   '+r'$X_{O_{2}}$', 'Shocked   '+r'$X_{NO}$', 'Shocked   '+r'$X_{O}$',
           'Reflected Pressure (kPa)', 'Reflected Temperature (K)', 'Reflected Density '+r'(kg.m$^{-3}$)', 'Reflected '+r'$\gamma$', 'Reflected Sound Speed (m/s)', 'Reflected Spec. Enthalpy '+r'(MJ.kg$^{-1}$)', 'Reflected Spec. Entropy '+r'(kJ.kg$^{-1}$.K$^{-1}$)' , 'Reflected Viscosity'+r' ($\mu$Pa.s)', 'Reflected Thermal Conductivity'+r'W.m$^{-1}$.K$^{-1}$', 'Reflected '+r'$X_{N_{2}}$', 'Reflected '+r'$X_{O_{2}}$', 'Reflected '+r'$X_{NO}$', 'Reflected '+r'$X_{O}$',
           'Reservoir Pressure (kPa)', 'Reservoir Temperature (K)', 'Reservoir Density '+r'(kg.m$^{-3}$)', 'Reservoir '+r'$\gamma$', 'Reservoir Sound Speed (m/s)', 'Reservoir Spec. Enthalpy '+r'(MJ.kg$^{-1}$)', 'Reservoir Spec. Entropy '+r'(kJ.kg$^{-1}$.K$^{-1}$)' , 'Reservoir Viscosity'+r' ($\mu$Pa.s)', 'Reservoir Thermal Conductivity'+r'W.m$^{-1}$.K$^{-1}$', 'Reservoir '+r'$X_{N_{2}}$', 'Reservoir '+r'$X_{O_{2}}$', 'Reservoir '+r'$X_{NO}$', 'Reservoir '+r'$X_{O}$']

label_y =  ['Incident Shock Speed','Incident Mach' ,'Driven Pressure', 
            'Shocked Pressure','Shocked Temperature','Shocked Density','Shocked Gamma','Shocked Sound Speed','Shocked Spec. Enthalpy','Shocked Spec. Entropy','Shocked Viscosity','Shocked Thermal Conductivity','Shocked XN2',        'Shocked xO2','Shocked XNO','Shocked xO',
            'Reflected Pressure','Reflected Temperature','Reflected Density','Reflected Gamma','Reflected Sound Speed','Reflected Spec. Enthalpy','Reflected Spec. Entropy','Reflected Viscosity','Reflected Thermal Conductivity','Reflected xN2','Reflected XO2','Reflected xNO','Reflected xO',
            'Reservoir Pressure','Reservoir Temperature','Reservoir Density','Reservoir Gamma','Reservoir Sound Speed','Reservoir Spec. Enthalpy','Reservoir Spec. Entropy','Reservoir Viscosity','Reservoir Thermal Conductivity','Reservoir xN2','Reservoir XO2','Reservoir xNO','Reservoir xO']

label_2 = ['Area Ratio', 'Pressure', 'Temperature', 'Density', 'Velocity', 'Mach',
           'Pitot Pressure', 'Fluorescence Lifetime', 'Characteristic Length', 'Viscosity', 'Mass Flow Rate',
           'Reynolds/m', 'Thermal Conductivity', 'Stand-off Distance', 'xN2','xO2','xNO','xO']


Format = [ r'%1.0f  ',  r'%1.2f  kPa',  r'%1.0f K',  r'%1.1f g/m3',
           r'%1.0f m/s',  r'%1.1f ',  r'%1.0f kPa',  r'%1.0f ns',
           r'%1.0f $\mu$m',  r'%1.0f $\mu$ Pa.s',  r'%1.2f g/s',  r'%1.0f 10$^5$/m',  
           r'%1.0f mW/(m.K)',  r'%1.1f %%', r'%1.1f %%',  r'%1.1f %%',  
           r'%1.1f %%',  r'%1.1f %%' ]

t1 = widgets.HTML(value="<b>Map Data</b>")    
t2 = widgets.HTML(value="<b>Parameters</b>")    
t3 = widgets.HTML(value="<b>.</b>")    

y     = widgets.Dropdown(description="Y Axis:",value=2,options=list_2, continuous_update=False,style=style)
z     = widgets.Dropdown(description="Attribute:",value=4,options=list_1, continuous_update=False,style=style)
cc    = widgets.Checkbox(value=True, description='Show Contour Levels', disabled=False, indent=True)
cl    = widgets.Checkbox(value=False, description='Show Contour Levels 2', disabled=False, indent=True)
cb    = widgets.Checkbox(value=False, description='Show Bars', disabled=False, indent=True)


nl    = widgets.FloatText(description="Number of Intervals:"  , value=30, continuous_update=False,style=style)
cmaps = widgets.Combobox( description="Colormap:",continuous_update=False, options=list_of_cmap, value="gist_earth",style=style )
shade = widgets.Combobox( description="Shading:",continuous_update=False, options=list_of_shading, value='gouraud',style=style )
iD1   = widgets.VBox([ t1,y, z])
iD2   = widgets.VBox([ t2,cc,cl,cb])
iD3   = widgets.VBox([ t3, nl, cmaps,shade])
UI_NozEx = widgets.HBox([ iD1, iD2,iD3 ])

Run = widgets.ToggleButtons(options=['Clear','Plot'],disabled=False, button_style='info')
UI_Run = widgets.HBox([Run])

def Input(y,z,cc,cl,cb,nl,cmaps,shade,Run2):
    if cl == True:
        Lvl = np.ones((3,int(nl)))
        Lvl[0,:] = np.exp( np.linspace(np.log(Fr_Exit[:,:,z].min()*1.1),np.log(Fr_Exit[:,:,z].max()*0.9),int(nl)))       
        Lvl[1,:] = np.exp( np.linspace(np.log(EF_Exit[:,:,z].min()*1.1),np.log(EF_Exit[:,:,z].max()*0.9),int(nl)))       
        Lvl[2,:] = np.exp( np.linspace(np.log(Eq_Exit[:,:,z].min()*1.1),np.log(Eq_Exit[:,:,z].max()*0.9),int(nl)))       
    if Run2=='Clear':
        #plt.clf()
        plt.close()
    if Run2=='Plot': 
        plt.figure('Mapss',figsize=[18,5])
        plt.subplot(1,3,1)
        plt.title('Nozzle Exit: Frozen Flow',fontsize=14)
        X, Y = np.meshgrid(Fr_Exit[0,:,0],C5[:,y])
        plt.pcolormesh( X, Y, Fr_Exit[:,:,z],   cmap=cmaps,  shading = shade)
        if cb == True: cbar = plt.colorbar()
        if cc == True: CS = plt.contour( X, Y , Fr_Exit[:,:,z],levels=int(nl), linewidths=0.7, colors='k');plt.clabel(CS,fmt=Format[z])
        if cl == True: CS = plt.contour( X, Y , Fr_Exit[:,:,z],levels=Lvl[0,:], linewidths=0.7, colors='k');plt.clabel(CS,fmt=Format[z])
        plt.xlabel('N.E. Area Ratio',fontsize=14)
        plt.ylabel(label_1[y],fontsize=14)
        plt.grid(alpha=0.5)
        plt.legend(title=label_2[z])

        plt.subplot(1,3,2)
        plt.title('Nozzle Exit:  Equilibrium to Frozen at Mach ='+str(M_EF),fontsize=14)
        X, Y = np.meshgrid(EF_Exit[0,:,0],C5[:,y])
        plt.pcolormesh( X, Y, EF_Exit[:,:,z],   cmap=cmaps,  shading = shade)
        if cb == True: cbar = plt.colorbar()
        if cc == True: CS = plt.contour( X, Y , EF_Exit[:,:,z], levels=int(nl), linewidths=0.7, colors='k'); plt.clabel(CS,use_clabeltext=1,fmt=Format[z])
        if cl == True: CS = plt.contour( X, Y , EF_Exit[:,:,z], levels=Lvl[1,:], linewidths=0.7, colors='k'); plt.clabel(CS,fmt=Format[z])
        plt.xlabel('N.E. Area Ratio',fontsize=14)
        plt.ylabel(label_1[y],fontsize=14)
        plt.grid(alpha=0.5)
        plt.legend(title=label_2[z])
        plt.subplot(1,3,3)

        plt.title('Nozzle Exit: Equilibrium Flow' ,fontsize=14)
        X, Y = np.meshgrid(Eq_Exit[0,:,0],C5[:,y])
        plt.pcolormesh( X, Y, Eq_Exit[:,:,z],   cmap=cmaps,  shading = shade)
        if cb == True: cbar = plt.colorbar()
        if cc == True: CS = plt.contour( X, Y , Eq_Exit[:,:,z], levels=int(nl), linewidths=0.7, colors='k');plt.clabel(CS,use_clabeltext=1,fmt=Format[z])
        if cl == True: CS = plt.contour( X, Y , Eq_Exit[:,:,z], levels=Lvl[2,:], linewidths=0.7, colors='k'); plt.clabel(CS,fmt=Format[z])
        plt.xlabel('N.E. Area Ratio',fontsize=14)
        plt.ylabel(label_1[y],fontsize=14)
        plt.grid(alpha=0.5)
        plt.legend(title=label_2[z])
        plt.tight_layout()
        #plt.show()
        #Run.value='Stop'
        
    return

Inp = widgets.interactive_output(Input,{ 'y':y,  'z':z, 'cc':cc, 'cl':cl ,'cb':cb,  'nl':nl, 'cmaps':cmaps,'shade':shade, 'Run2':Run } )
UI = widgets.VBox([UI_NozEx,UI_Run])
display(UI, Inp)

Output()

#### Nozzle Exit Results

Analogously to the map above, the following code creates a data table for a given nozzle exit parameter. The user must select the Y_Axis parameter from the list of shock tube parameters (see below), and select Attribute from a list of nozzle exit output (see below). 


Y_Axis labels:
    - Incident Shock Speed, Incident Mach, Driven Pressure, Shocked Pressure, Shocked Temperature, Shocked Density, Shocked Gamma, Shocked Sound Speed, Shocked Spec. Enthalpy, Shocked Spec. Entropy, Shocked Viscosity, Shocked Thermal Conductivity, Shocked XN2, Shocked xO2, Shocked XNO, Shocked xO, Reflected Pressure, Reflected Temperature, Reflected Density, Reflected Gamma, Reflected Sound Speed, Reflected Spec. Enthalpy, Reflected Spec. Entropy, Reflected Viscosity, Reflected Thermal Conductivity, Reflected xN2, Reflected XO2, Reflected xNO, Reflected xO, Reservoir Pressure, Reservoir Temperature, Reservoir Density, Reservoir Gamma, Reservoir Sound Speed, Reservoir Spec. Enthalpy, Reservoir Spec. Entropy, Reservoir Viscosity, Reservoir Thermal Conductivity, Reservoir xN2, Reservoir XO2, Reservoir xNO, Reservoir xO


Attribute labels:
     - Area Ratio, Pressure, Temperature, Density, Velocity, Mach, Pitot Pressure, Fluorescence Lifetime, Characteristic Length, Viscosity, Mass Flow Rate, Reynolds/m, Thermal Conductivity, Stand-off Distance, xN2, xO2, xNO, xO

In [57]:
Y_Axis    = 'Driven Pressure' #The
Attribute = 'Fluorescence Lifetime' 

Dic = dict()
for i in np.arange(len(label_2)):
    Dic[label_2[i]] = [np.round(Fr_Exit[:,:,i],2),np.round(EF_Exit[:,:,i],2),np.round(Eq_Exit[:,:,i],2)]

    
Dic2 = dict()
for i in np.arange(len(label_1)):
    Dic2[label_y[i]] = [np.round(C5[:,i],2),i]

    
Name_x = ['Nozzle-Exit Area Ratio :']*(NE_AR.size)    
col    = pd.MultiIndex.from_arrays([ Name_x,  np.round(NE_AR,2) ])
row    = pd.MultiIndex.from_arrays([Dic2[Y_Axis][0]])

m = []
for c in [0,1,2]: m.append( pd.DataFrame(Dic[Attribute][c], columns=col, index=row) )
display(widgets.HTML( value='<b>Output: </b>' + Attribute) )
pd.concat( [m[0],m[1],m[2]], keys=[ "Frozen Flow" , "Eq. to Frozen Flow" , "Equilibrium Flow" ], names = [label_1[Dic2[Y_Axis][1]]+':'] )


HTML(value='<b>Output: </b>Fluorescence Lifetime')

Nozzle-Exit Area Ratio :                       \
                                                10.00  41.05  72.11  103.16   
Driven Pressure (kPa):                                                        
Frozen Flow            10.00                      1.12   6.20  11.84  17.62   
                       14.74                      1.19   6.56  12.51  18.58   
                       19.47                      1.25   6.85  13.04  19.34   
                       24.21                      1.29   7.07  13.44  19.91   
                       28.95                      1.32   7.22  13.73  20.32   
                       33.68                      1.34   7.33  13.92  20.61   
                       38.42                      1.35   7.38  14.03  20.75   
                       43.16                      1.36   7.43  14.11  20.87   
                       47.89                      1.36   7.45  14.15  20.93   
                       52.63                      1.36   7.45  14.16  20.94   
                       57.37                      1.36   7.43  14.11  20.88   
                       62.11                      1.35   7.39  14.04  20.77   
                       66.84                      1.34   7.34  13.96  20.66   
                       71.58                      1.33   7.30  13.87  20.53   
                       76.32                      1.32   7.24  13.76  20.38   
                       81.05                      1.31   7.18  13.66  20.23   
                       85.79                      1.30   7.12  13.55  20.07   
                       90.53                      1.28   7.06  13.43  19.90   
                       95.26                      1.27   6.99  13.30  19.72   
                       100.00                     1.26   6.92  13.17  19.54   
Eq. to Frozen Flow     10.00                      1.26   6.97  13.28  19.69   
                       14.74                      1.35   7.41  14.08  20.84   
                       19.47                      1.41   7.72  14.66  21.66   
                       24.21                      1.45   7.93  15.04  22.20   
                       28.95                      1.47   8.05  15.27  22.53   
                       33.68                      1.48   8.12  15.39  22.71   
                       38.42                      1.48   8.13  15.41  22.73   
                       43.16                      1.48   8.13  15.41  22.73   
                       47.89                      1.48   8.11  15.37  22.68   
                       52.63                      1.47   8.07  15.30  22.58   
                       57.37                      1.46   8.00  15.18  22.42   
                       62.11                      1.44   7.92  15.04  22.21   
                       66.84                      1.43   7.85  14.89  22.01   
                       71.58                      1.41   7.76  14.74  21.79   
                       76.32                      1.40   7.68  14.58  21.57   
                       81.05                      1.38   7.59  14.43  21.35   
                       85.79                      1.37   7.51  14.28  21.13   
                       90.53                      1.35   7.42  14.11  20.90   
                       95.26                      1.33   7.33  13.95  20.66   
                       100.00                     1.32   7.24  13.79  20.43   
Equilibrium Flow       10.00                      1.84  11.19  21.38  31.50   
                       14.74                      1.90  11.05  21.11  31.15   
                       19.47                      1.90  10.87  20.79  30.74   
                       24.21                      1.87  10.68  20.46  30.28   
                       28.95                      1.84  10.47  20.11  29.79   
                       33.68                      1.80  10.28  19.76  29.31   
                       38.42                      1.76  10.07  19.39  28.77   
                       43.16                      1.73   9.90  19.08  28.34   
 